In [29]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import time
import numpy as np
from sklearn.model_selection import KFold

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

In [14]:
def data_preprocessing(path, test=False):
    #start_time = time.time()
    
    #create appropriate file path
    if test == False:
        pfilename = path + "/product_training.json"
        rfilename = path + "/review_training.json"
    else:
        pfilename = path + "/product_test.json"
        rfilename = path + "/review_test.json"
    
    #extract files as pandas dataframes
    product_df = pd.read_json(pfilename)
    
    review_df = pd.read_json(rfilename).drop_duplicates(subset=["reviewerID", "unixReviewTime"], keep="first")
    ## 11.66 seconds to get to here
    
    review_df.drop(columns=["reviewerID","vote", "unixReviewTime","reviewTime","style","reviewerName","image"], axis=1 ,inplace=True)
    
    review_df['reviewText'].fillna("", inplace=True)
    review_df['summary'].fillna("", inplace=True)
    
    review_df.sort_values('asin', inplace = True)
    product_df.sort_values('asin', inplace = True)
    
    group = review_df.groupby("asin")
    
    #review_group_df = pd.DataFrame(columns = ['asin', 'numReviews', 'percentVerified','reviewText','summaryText', 'awesomeness'])
    
    # about the same amount of time to get to here
    start_time = time.time()
    datalist = []
    count = 0
    #awesome_pos = 0
    for asin, data in group:
        verifiedCount = data['verified'].sum()
        reviewCount = data['asin'].count()
        percentVerified = verifiedCount / reviewCount
        reviewText = ' '.join(data['reviewText'])
        #reviewText = ' '.join(transform_document(x) for x in data['reviewText'])
        #summaryText = ""
        summaryText = ' '.join(data['summary'])
        #summaryText = ' '.join(transform_document(x) for x in data['summary'])
        #reviewText = transform_document(' '.join(data['reviewText']))
        #summaryText = transform_document(' '.join(data['summary']))
        #awesomeness = 0
        while (product_df['asin'][count] != asin):
               count = count + 1
        
        awesomeness = product_df['awesomeness'][count]
        #awesome_pos = awesome_pos + reviewCount
        #awesomeness = product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0] #might be slow
        datalist.append([asin,  reviewCount, percentVerified, reviewText, summaryText, awesomeness])
        
        count = count + 1
        #if count > 100:
        #    break
        
        '''new_row = {'asin': asin, 
                   'numReviews': reviewCount, 
                   'percentVerified': percentVerified, 
                   'reviewText': transform_document(' '.join(data['reviewText'])), 
                   'summaryText': transform_document(' '.join(data['summary'])), 
                   'awesomeness': product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0]} 
        review_group_df = review_group_df.append(new_row, ignore_index = True)
         '''
    review_group_df = pd.DataFrame(datalist,columns =['asin', 'numReviews', 'percentVerified','reviewText','summaryText', 'awesomeness'])    
    
    review_group_df.to_json(r'../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data.json')
    end_time = time.time()
    print(end_time - start_time)
    
    return review_group_df
    

In [15]:
# Preprocessed Data Generated (Reviews and Summaries aggregated, no NLP processing)
review_group_df = data_preprocessing("../devided_dataset_v2/CDs_and_Vinyl/train")
#review_group_df.head()
#review_group_df

23.382253885269165


In [62]:
# Pipeline based on countvectorizer, tfidf, multinomial NB (can try support vector machine or decision tree also)
# will want to use the k-fold cross-validation on this
# will want to figure out how to combine the tfidf vectors for the review and summary with the other features we had in mind
# e.g., the percent of verified reviews
#text_clf = Pipeline([('vect', make_column_transformer((CountVectorizer(), ['reviewText']), (CountVectorizer(), ['summaryText'])) ), 
#                     ('tfidf', make_column_transformer((TfidfTransformer(), [0]), (TfidfTransformer(), ['summaryText'])) ), 
#                     ('clf', MultinomialNB())] )
#text_clf.fit(review_group_df['reviewText'] ,review_group_df['awesomeness'])

In [63]:
text_clf.fit(review_group_df, review_group_df['awesomeness'])

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [48]:
# sanity check - if the model can't predict the results for the training data, then it really didn't work
predicted_reviews = text_clf.predict(review_group_df['reviewText'])

In [49]:
np.mean(predicted_reviews == review_group_df['awesomeness'])

0.6630697622408901

In [32]:
# Setting up the Text Processing (Count Vectorizer and TFIDF Transform)
countvect = CountVectorizer()

In [50]:
reviewTrainCounts = countvect.fit_transform(review_group_df['reviewText'])
summaryTrainCounts = countvect.fit_transform(review_group_df['summaryText'])

In [51]:
#reviewTrainCounts.shape
summaryTrainCounts.shape

(71543, 55093)

In [53]:
reviewTransformer = TfidfTransformer(use_idf=True).fit(reviewTrainCounts)
summaryTransformer = TfidfTransformer(use_idf=True).fit(summaryTrainCounts)
reviewTrain_tfidf = reviewTransformer.transform(reviewTrainCounts)
summaryTrain_tfidf = summaryTransformer.transform(summaryTrainCounts)

summaryTrain_tfidf.shape
#reviewTrain_tfidf.shape

(71543, 55093)

In [55]:
reviewTrain_tfidf

<71543x311376 sparse matrix of type '<class 'numpy.float64'>'
	with 25824726 stored elements in Compressed Sparse Row format>

In [57]:
clf = MultinomialNB().fit(hstack(reviewTrain_tfidf, summaryTrain_tfidf), review_group_df['awesomeness'])

ValueError: blocks must be 2-D

PSEUDOCODE FOR WHAT WE NEED TO DO WITH THE TRAINING DATA:
 - Saahir/Amy starts by putting all of the reviews for a product in one string. This string is the DOCUMENT for the reviews of a product
 - I will then take the corpus and transform it to remove stopwords, punctuation, and lemmatize everything. This is the true "bag of words".
 - Then, I use the cleaned text data to train the Naive Bayes classifier
 - When we get test data, first clean in the same way, then use it in the classifier

In [7]:
corpus = ["I am being handed a list of documents", "Each of these documents has several unique words", "The words will represent the class of each review", "I am also removing stopwords in order to make this make more sense"]
cleaned_corpus = [transform_document(doc) for doc in corpus]
vocabulary = vocabulary_from_corpus(cleaned_corpus, True)
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)), 
                 ('tfid', TfidfTransformer())]).fit(cleaned_corpus)),

In [ ]:
product_df.loc[product_df['asin'] == '0000B049F5B33CD310EB1AB236E20191', 'awesomeness'].values[0]

NLP Functions

In [12]:
def get_stopwords():
    file = open('en.txt')
    stopwords = []
    for line in file:
        stopwords.append(line.rstrip())
    return stopwords

In [23]:
def transform_corpus(review_group):
    stopwords = get_stopwords()
    reviewTextSet = review_group['reviewText']
    for index in review_group.index:
        curr_parsed = nlp(reviewTextSet[index].lower())
        doclist = []
        for token in curr_parsed:
            lemma = token.lemma_
            if not(re.match("[a-z0-9]+", lemma)):
                continue
            if lemma not in stopwords:
                doclist.append(lemma)
        reviewTextSet[index] = " ".join(doclist)

In [197]:
def transform_document(doc, remove_stopwords = True):
    #new_doc = ""
    stopwords = get_stopwords() # is this slow?
    parsed_text = nlp(doc) # is this slow
    doclist = []
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            doclist.append(lemma) # this is less slow?
    return " ".join(doclist)

In [198]:
%timeit transform_document("First, you need to preprocess the raw text data. This may involve tasks like tokenizing the text (i.e., splitting it into individual words), removing stopwords, stemming or lemmatizing the words, and converting the text into a numerical format that can be used as input for the model. Then, you need to split the data into training and testing sets. The training set will be used to train the model, while the testing set will be used to evaluate its performance.")

15.1 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
def bag_of_words(review_text, remove_stopwords = True):
    word_bag = {}
    stopwords = get_stopwords()
    parsed_text = nlp(review_text)
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            if lemma in word_bag:
                word_bag[lemma] += 1
            else:
                word_bag[lemma] = 1
    return word_bag

In [51]:
def vocabulary_from_corpus(corpus, remove_stopwords = True):
    vocab_set = set()
    for document in corpus:
        word_bag = bag_of_words(document, remove_stopwords)
        for word in word_bag.keys():
            vocab_set.add(word)
    return list(vocab_set)
vocabulary_from_corpus(['this is the first document', 'this document is the second document', 'and this is the third one', 'is this the first document'], False)

['first', 'the', 'second', 'third', 'be', 'and', 'this', 'one', 'document']

In [88]:
product_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/product_training.json')
review_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/review_training.json')
#len(product_df.index)

In [6]:
teststring = "First, you need to preprocess the raw text data. This may involve tasks like tokenizing the text (i.e., splitting it into individual words), removing stopwords, stemming or lemmatizing the words, and converting the text into a numerical format that can be used as input for the model. Then, you need to split the data into training and testing sets. The training set will be used to train the model, while the testing set will be used to evaluate its performance."
#teststring.lower()
parsed = nlp(teststring.lower())

In [46]:
string_features = ['reviewText', 'summaryText']
string_transformer = Pipeline(
    [('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer())]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("str", string_transformer, string_features)
    ], remainder='passthrough'
)


clf = Pipeline(steps = [("preprocess", preprocessor), ('classifier', LogisticRegression())])
x = review_group_df.loc[:, review_group_df.columns != 'awesomeness']
y = review_group_df.loc[:, review_group_df.columns == 'awesomeness']


kf = KFold(n_splits = 10, shuffle = True)
for i, (train_index, test_index) in enumerate(kf.split(x)):
    #print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")
    x_train = x.loc[train_index, :]
    x_test = x.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]
    
    #print(x_train.shape)
    #print(y_train.shape)
    clf = Pipeline(steps = [("preprocess", preprocessor), ('classifier', LogisticRegression()) ])
    clf.fit(x_train, y_train)
    #print(x_train)
    #print(y_train.shape)
    #X_trans = preprocessor.fit_transform(x_train)
    #print(x_train.shape)
    print("model score: %.3f" % clf.score(X_test, y_test))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2 and the array at index 1 has size 64388